# 기초 작업

### 모듈 불러오기

In [1]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
import re
import random
import sqlite3
from pymongo import MongoClient

### 크롬드라이버가 url 접속하도록 하기

In [2]:
# 창 실행 옵션x
options = wd.ChromeOptions()
options.add_argument("headless") 

# 크롬드라이버 열기
driver = wd.Chrome('chromedriver', chrome_options=options) # 크롬드라이버 경로

# 드라이버가 해당 url 접속
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)


<ipython-input-2-264a2b05d9a8>:6: DeprecationWarning: use options instead of chrome_options
  driver = wd.Chrome('chromedriver', chrome_options=options) # 크롬드라이버 경로


# 함수 만들기_1

### 기존 함수

In [3]:
# 연대선택 era_num=1 : 2020년대, era_num=2 : 2010년대
def click_era(era_num) :    
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li['+str(era_num)+']/span/label').click()
    time.sleep(0.1)
    
# 연도선택 year_num : 최근 -> 과거
def click_year(year_num) :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li['+str(year_num)+']/span/label').click()
    time.sleep(0.1)

# 월간선택 
def click_month(month_year) :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li['+str(month_year)+']/span/label').click()
    time.sleep(0.1)

# 주간 선택
def click_week(week_num) :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li['+str(week_num)+']/span/label').click()
    time.sleep(0.1)

# 장르선택 ('장르 종합'으로 선택)
def click_genre() :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
    time.sleep(0.1)
    
    
# 검색버튼 클릭
def click_search() :
    driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()
    time.sleep(0.1)
    
    

# 연대선택 개수 찾기
def find_era_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth1.view > div.list_value > ul > li'))
    return length    
    
# 연도선택 개수 찾기
def find_year_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth2.view > div.list_value > ul > li'))
    return length

# 월간선택 개수 찾기
def find_month_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth3.view > div.list_value > ul > li'))
    return length

# 주간선택 개수 찾기
def find_week_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth4.view > div.list_value > ul > li'))
    return length

### 곡 id 리스트 만드는 함수 만들기

In [4]:
# 리스트 만들기
def get_song_id_list() :
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    id_list=[]
    for id in soup.find_all('input',{'name':'input_check'}):
        id_list.append(id.get('value'))
            
    return id_list

# 크롤링_1 : 곡 ID 가져오기

### 곡 id 리스트 만들기 (중복 제거)

In [5]:
# 리스트 만들기
song_id_list_all =[]

# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

# 주간차트 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click()
time.sleep(0.1)

# 나머지 멈추는 시기 결정
stop_era = 2
stop_year = 2
stop_month = 6

# 나머지 자동 선택
len_era = find_era_length()
for i in range(1,len_era+1) :   
    click_era(i)
        
    len_year = find_year_length()
    for j in range(1,len_year+1) :
        click_year(j)
                
        len_month = find_month_length()
        
        if i == stop_era and j == stop_year :
            for k in range(stop_month, len_month+1) :
                click_month(k)

                len_week = find_week_length()
                for l in range(1, len_week+1) :
                    click_week(l)
                    click_genre()
                    click_search()
                        
                    song_id_list_all += get_song_id_list()
        
        else :
            for k in range(1, len_month+1) :
                click_month(k)

                len_week = find_week_length()
                for l in range(1, len_week+1) :
                    click_week(l)
                    click_genre()
                    click_search()

                    song_id_list_all += get_song_id_list()
            
        if i == stop_era and j == stop_year : break    
                            
    if i == stop_era : break;

# 드라이버 종료
driver.quit()        

# 중복 제거
song_id_list = list(set(song_id_list_all))

print("전체 곡 수 : ", len(song_id_list_all))
print("중복이 제거된 전체 곡 수 : ", len(song_id_list))

전체 곡 수 :  15900
중복이 제거된 전체 곡 수 :  1137


# 함수 만들기_2

### 곡 정보 가져오는 함수 만들기

In [6]:
def get_song_info(song_id_num) :
    
    song_id =  song_id_num
    
    # 주소 입력
    url = 'https://www.melon.com/song/detail.htm?songId='+str(song_id)
    
    # 차단 방지
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    
    # 접속
    req = requests.get(url, headers=headers)
    source = BeautifulSoup(req.text, 'lxml')
    
    # 곡명 가져오기
    song_name = source.find_all('div', class_ = 'song_name')[0].text
    song_name = song_name[10:].split()
    song_name = " ".join(song_name)

    # 가수 이름 가져오기
    artist_name_source =  source.select('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)')
    artist_name_list = []
    for i in range(len(artist_name_source)) :
        artist_name_list.append(artist_name_source[i].get_text())
    artist_name = ', '.join(artist_name_list)
    
    # 앨범 가져오기
    album_name = source.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(2) > a').get_text()

    # 발매일 가져오기
    release_date = source.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').get_text()
    # yyyy.mm.dd -> yyyy-mm-dd
    release_date = release_date.replace('.', '-')

    # 장르 가져오기
    genre = source.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').get_text()
    

    return song_name, artist_name, album_name, release_date, genre
    

# 크롤링_2 : 곡 정보 가져오기

### 각각의 정보를 list에 저장

In [7]:
print("총 id 개수 : ", len(song_id_list))
print()

song_name_list = []
artist_name_list = []
album_name_list = []
release_date_list = []
genre_list = []

print("시작 :")
i = 1
for id in song_id_list :
    song_name, artist_name, album_name, release_date, genre = get_song_info(id)
    
    # 차단 방지
    # time.sleep(random.uniform(0, 0.5))
    
    song_name_list.append(song_name)
    artist_name_list.append(artist_name)
    album_name_list.append(album_name)
    release_date_list.append(release_date)
    genre_list.append(genre)
    
    
    print(i," ", end = '')
    i += 1
    
    # if i == 400 or i == 800 :
    #     time.sleep(120)


print()
print("끝")


총 id 개수 :  1137

시작 :
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  2

### 데이터프레임으로 저장

In [8]:
df_song_info = pd.DataFrame({"song_id" : song_id_list, "song_name" : song_name_list,
                             "artist_name" : artist_name_list, "album_name" : album_name_list, 
                             "release_date" : release_date_list, "genre" : genre_list})

df_song_info

,song_id,song_name,artist_name,album_name,release_date,genre
0,32163199,낙서 (Feat. 백예린) (Prod. by WOOGIE),펀치넬로 (punchnello),낙서,2019-11-11,랩/힙합
1,31399725,닿은 순간 (Ooh La La La),EXO,DON'T MESS UP MY TEMPO - The 5th Album,2018-11-02,댄스
2,32217924,춤 (Groove),EXO,OBSESSION - The 6th Album,2019-11-27,댄스
3,711627,죄와벌,SG 워너비,살다가,2005-03-23,발라드
4,31133898,너 없인 안 된다,비투비,THIS IS US,2018-06-18,발라드
...,...,...,...,...,...,...
1132,31417871,SOLO,제니 (JENNIE),SOLO,2018-11-12,랩/힙합
1133,31584299,잊지마요 (Feat. 박지훈),하성운,잊지마요,2019-01-28,발라드
1134,32872978,Dynamite,방탄소년단,Dynamite (DayTime Version),2020-08-24,댄스
1135,33265558,흑백사진 (2021) (바른연애 길잡이 X KCM),KCM,흑백사진 (2021) (바른연애 길잡이 X KCM),2021-02-06,발라드


### CSV로 저장

In [9]:
df_song_info.to_csv('csv/song_info/song_info.csv', encoding = 'utf-8-sig', index = False)

### SQLite3 DB로 저장

In [10]:
con = sqlite3.connect("db/song.db")

df_song_info.to_sql("song_info", con, if_exists="replace", index=False)

con.close()

### SQLite3 DB 저장 확인

In [11]:
con = sqlite3.connect("db/song.db")

df = pd.read_sql("SELECT * FROM song_info", con)

con.close()

df

,song_id,song_name,artist_name,album_name,release_date,genre
0,32163199,낙서 (Feat. 백예린) (Prod. by WOOGIE),펀치넬로 (punchnello),낙서,2019-11-11,랩/힙합
1,31399725,닿은 순간 (Ooh La La La),EXO,DON'T MESS UP MY TEMPO - The 5th Album,2018-11-02,댄스
2,32217924,춤 (Groove),EXO,OBSESSION - The 6th Album,2019-11-27,댄스
3,711627,죄와벌,SG 워너비,살다가,2005-03-23,발라드
4,31133898,너 없인 안 된다,비투비,THIS IS US,2018-06-18,발라드
...,...,...,...,...,...,...
1132,31417871,SOLO,제니 (JENNIE),SOLO,2018-11-12,랩/힙합
1133,31584299,잊지마요 (Feat. 박지훈),하성운,잊지마요,2019-01-28,발라드
1134,32872978,Dynamite,방탄소년단,Dynamite (DayTime Version),2020-08-24,댄스
1135,33265558,흑백사진 (2021) (바른연애 길잡이 X KCM),KCM,흑백사진 (2021) (바른연애 길잡이 X KCM),2021-02-06,발라드


### MongoDB 저장

In [12]:
client = MongoClient('localhost', 27017)
db = client['song']
col = db['song_info']

df_dictionarys = [{**row.dropna().to_dict()} for index,row in df_song_info.iterrows()]

col.insert_many(df_dictionarys)